In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from IPython.core.pylabtools import figsize

In [2]:
rawtext = open("/media/joe/HD/data/gutenberg/alice_in_wonderland.txt", "r").read()[710:]
len(rawtext)

172656

In [3]:
split = int(0.9*len(rawtext))
trainraw = rawtext.lower()[:split]
testraw = rawtext.lower()[split:]

In [4]:
chars = sorted(list(set(rawtext.lower())))
charmap = {chars[i]:i for i in range(len(chars))}
len(chars)

60

In [5]:
seq_len = 25

def train_inpt_fn():
    def _gen():
        while True:
            seq_start = np.random.randint(0, len(trainraw)-seq_len-1)
            sequence = trainraw[seq_start:seq_start+seq_len]
            yield {"x":np.array([charmap[c] for c in sequence], dtype=np.int32)}, \
            charmap[trainraw[seq_start+seq_len]]
            
    ds = tf.data.Dataset.from_generator(_gen, ({"x":tf.int32}, tf.int32), 
                                        ({"x":[seq_len,]}, []))
    ds = ds.batch(1024)
    ds = ds.prefetch(1)
    return ds.make_one_shot_iterator().get_next()

def test_inpt_fn():
    def _gen():
        for i in range(1000):
            seq_start = i*seq_len
            sequence = testraw[seq_start:seq_start+seq_len]
            yield {"x":np.array([charmap[c] for c in sequence], dtype=np.int32)}, \
                  charmap[testraw[seq_start+seq_len]]
            
    ds = tf.data.Dataset.from_generator(_gen, ({"x":tf.int32}, tf.int32), 
                                        ({"x":[seq_len,]}, []))
    ds = ds.batch(512)
    ds = ds.prefetch(1)
    return ds.make_one_shot_iterator().get_next()

In [6]:
train_inpt_fn()

({'x': <tf.Tensor 'IteratorGetNext:0' shape=(?, 25) dtype=int32>},
 <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=int32>)

In [7]:
def model_fn(features, labels, mode, params):
    
    x_oh = tf.one_hot(features["x"], params["num_chars"])
    if labels is not None:
        y_oh = tf.one_hot(labels, params["num_chars"])

    if params["lstm"]:
        cell = tf.nn.rnn_cell.LSTMCell(params["hidden"])
    else:
        cell = tf.nn.rnn_cell.BasicRNNCell(params["hidden"])
    outputs, state = tf.nn.dynamic_rnn(cell, x_oh, dtype=tf.float32)

    net = outputs[:, -1, :]
    
    is_training = mode == tf.estimator.ModeKeys.TRAIN

    net = tf.layers.dense(net, params["hidden"], activation=tf.nn.relu)
    net = tf.layers.dropout(net, 0.5, training=is_training)

    logits = tf.layers.dense(net, params["num_chars"])
    probs = tf.nn.softmax(logits)
    predictions = tf.argmax(probs, -1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        default = tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY
        return tf.estimator.EstimatorSpec(mode=mode, 
                            predictions={"probs":probs},
                            export_outputs={default:tf.estimator.export.PredictOutput(
                                {"probs":probs, "state":state[0]}
                            )})
                            #export_outputs={"probs":tf.estimator.export.PredictOutput(probs),
                            #               "state":tf.estimator.export.PredictOutput(state[0]),
                            #               default:tf.estimator.export.PredictOutput(probs)})
    
    loss = tf.losses.softmax_cross_entropy(y_oh, logits)
    accuracy = tf.metrics.accuracy(labels, predictions)
    metrics = {"accuracy":accuracy}
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode=mode,
                            loss=loss, eval_metric_ops=metrics)
    
    gs = tf.train.get_or_create_global_step()
    #optimizer = tf.train.MomentumOptimizer(1e-3, 0.9)
    optimizer = tf.train.AdamOptimizer(1e-3)
    
    train_op = optimizer.minimize(loss, global_step=gs)
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, 
                        train_op=train_op, eval_metric_ops=metrics)

In [8]:
params={"num_chars":len(chars), "lstm":True, "hidden":256}
mod = "/media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256/"
model = tf.estimator.Estimator(model_fn, mod, 
                               params=params)

model.train(lambda: train_inpt_fn(), steps=10000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7f642ca4a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256/model.ckpt-10000
INF

INFO:tensorflow:loss = 0.380033, step = 16800 (17.644 sec)
INFO:tensorflow:global_step/sec: 5.78306
INFO:tensorflow:loss = 0.350297, step = 16900 (17.294 sec)
INFO:tensorflow:global_step/sec: 5.62241
INFO:tensorflow:loss = 0.385118, step = 17000 (17.784 sec)
INFO:tensorflow:global_step/sec: 5.47547
INFO:tensorflow:loss = 0.336469, step = 17100 (18.264 sec)
INFO:tensorflow:global_step/sec: 5.42355
INFO:tensorflow:loss = 0.39373, step = 17200 (18.439 sec)
INFO:tensorflow:global_step/sec: 5.63157
INFO:tensorflow:loss = 0.343755, step = 17300 (17.757 sec)
INFO:tensorflow:global_step/sec: 5.64462
INFO:tensorflow:loss = 0.331682, step = 17400 (17.717 sec)
INFO:tensorflow:global_step/sec: 5.19713
INFO:tensorflow:loss = 0.297558, step = 17500 (19.240 sec)
INFO:tensorflow:global_step/sec: 5.14657
INFO:tensorflow:loss = 0.297503, step = 17600 (19.430 sec)
INFO:tensorflow:global_step/sec: 4.16848
INFO:tensorflow:loss = 0.358647, step = 17700 (24.000 sec)
INFO:tensorflow:global_step/sec: 4.66391
I

In [9]:
def serving_input_fn():
    features = {"x":tf.placeholder(tf.int32, [1, seq_len])}
    return tf.estimator.export.build_raw_serving_input_receiver_fn(features, None)

In [10]:
model.export_savedmodel("/media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256_export/", serving_input_fn())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256/model.ckpt-20000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256_export/temp-b'1537322950'/saved_model.pb


b'/media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256_export/1537322950'

In [11]:
params={"num_chars":len(chars), "lstm":True, "hidden":256}
mod = "/media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256_adam/"
model = tf.estimator.Estimator(model_fn, mod, 
                               params=params)

model.train(lambda: train_inpt_fn(), steps=25000)
model.export_savedmodel("/media/joe/HD/text_pred_models/alice_lstm_256_adam_export/", serving_input_fn())

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256_adam/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff4f828def0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints

INFO:tensorflow:global_step/sec: 5.18506
INFO:tensorflow:loss = 0.824451, step = 7100 (19.285 sec)
INFO:tensorflow:global_step/sec: 5.4589
INFO:tensorflow:loss = 0.840783, step = 7200 (18.318 sec)
INFO:tensorflow:global_step/sec: 5.54296
INFO:tensorflow:loss = 0.825914, step = 7300 (18.042 sec)
INFO:tensorflow:global_step/sec: 5.3646
INFO:tensorflow:loss = 0.774246, step = 7400 (18.642 sec)
INFO:tensorflow:global_step/sec: 5.28235
INFO:tensorflow:loss = 0.761384, step = 7500 (18.929 sec)
INFO:tensorflow:global_step/sec: 5.2092
INFO:tensorflow:loss = 0.714292, step = 7600 (19.198 sec)
INFO:tensorflow:global_step/sec: 5.34642
INFO:tensorflow:loss = 0.791309, step = 7700 (18.704 sec)
INFO:tensorflow:global_step/sec: 5.44468
INFO:tensorflow:loss = 0.765913, step = 7800 (18.372 sec)
INFO:tensorflow:global_step/sec: 5.31488
INFO:tensorflow:loss = 0.662816, step = 7900 (18.812 sec)
INFO:tensorflow:global_step/sec: 5.4367
INFO:tensorflow:loss = 0.7649, step = 8000 (18.394 sec)
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 5.20734
INFO:tensorflow:loss = 0.407981, step = 15100 (19.204 sec)
INFO:tensorflow:global_step/sec: 4.97977
INFO:tensorflow:loss = 0.417983, step = 15200 (20.081 sec)
INFO:tensorflow:global_step/sec: 4.86606
INFO:tensorflow:loss = 0.405588, step = 15300 (20.551 sec)
INFO:tensorflow:Saving checkpoints for 15319 into /media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256_adam/model.ckpt.


KeyboardInterrupt: 

In [12]:
model.export_savedmodel("/media/joe/HD/text_pred_models/alice_lstm_256_adam_export/", serving_input_fn())

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['probs', 'serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /media/joe/HD/text_pred_models/alice_in_wonderland_lstm_256_adam/model.ckpt-15319
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /media/joe/HD/text_pred_models/alice_lstm_256_adam_export/temp-b'1536116411'/saved_model.pb


b'/media/joe/HD/text_pred_models/alice_lstm_256_adam_export/1536116411'